In [7]:
import pandas as pd
import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from datetime import datetime
import seaborn as sns
import math
import matplotlib.pyplot as plt
import numpy as np
import folium
import json

## Accès API Strava

In [2]:
auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"

# Obtenir la date actuelle
creation_date = datetime.now().strftime('%Y-%m-%d')

payload = {
    'client_id': "114062",
    'client_secret': "3ff69d1668764ef3f5ebdfbe71ecc04bf694d74c",
    'refresh_token': "e4a08d7904a3eedba9782ee6aa7b8644898beaff",
    'grant_type': "refresh_token",
    'f': 'json'
}

print("Requesting Token...\n")
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']
print("Access Token = {}\n".format(access_token))

header = {'Authorization': 'Bearer ' + access_token}
#param = {'per_page': 200, 'page': 1}
#my_dataset = requests.get(activites_url, headers=header, params=param).json()
# Liste pour stocker toutes les activités
all_activities = []

# Boucle pour récupérer toutes les pages d'activités
page = 1
while True:
    param = {'per_page': 200, 'page': page}
    activities = requests.get(activites_url, headers=header, params=param).json()

    # Si aucune activité n'est retournée, on arrête la boucle
    if not activities:
        break

    # Ajout des activités à la liste
    all_activities.extend(activities)

    # Incrémentation du numéro de page
    page += 1

# Conversion en DataFrame pandas
activities_df = pd.DataFrame(all_activities)

# Sauvegarde en fichier CSV
csv_file_path = f'../data/strava_activities_raw_{creation_date}.csv'
activities_df.to_csv(csv_file_path, index=False)

print(f"Les données ont été sauvegardées dans {csv_file_path}")


Requesting Token...

Access Token = bbfbf712a6a7e68fa3bd89247f061ddf7ce4b82c

Les données ont été sauvegardées dans ../data/strava_activities_raw_2025-07-24.csv


## Refresh quotidien Données Strava ?

## Data exploration

In [ ]:
activities_df.head(2)

,resource_state,athlete,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,...,elev_high,elev_low,upload_id,upload_id_str,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,average_cadence
0,2,"{'id': 40727595, 'resource_state': 1}",Big moustache ?,37258.4,5694,6052,283.0,Ride,Ride,10.0,...,141.0,57.4,1.624303e+10,16243025075,garmin_ping_462010212174,False,3,3,False,NaN
1,2,"{'id': 40727595, 'resource_state': 1}",Course à Dwight dans le Schrute,10017.0,2890,2893,75.0,Run,Run,0.0,...,90.4,56.2,1.622995e+10,16229954716,garmin_ping_461653472390,False,0,1,False,73.2


In [5]:
activities_df[["map"]]

,map
0,"{'id': 'a15219932005', 'summary_polyline': '{f..."
1,"{'id': 'a15210086165', 'summary_polyline': 'ci..."
2,"{'id': 'a15197835902', 'summary_polyline': 's|..."
3,"{'id': 'a15184146738', 'summary_polyline': 's~..."
4,"{'id': 'a15163453200', 'summary_polyline': 'wv..."
...,...
608,"{'id': 'a2596458576', 'summary_polyline': '{gp..."
609,"{'id': 'a2589310060', 'summary_polyline': 'mho..."
610,"{'id': 'a2583989594', 'summary_polyline': '{mg..."
611,"{'id': 'a2332672484', 'summary_polyline': 'sip..."


In [8]:
activities_df["map"] = activities_df["map"].apply(json.dumps)


In [11]:
activities_df[["map"]].info

<bound method DataFrame.info of                                                    map
0    {"id": "a15219932005", "summary_polyline": "{f...
1    {"id": "a15210086165", "summary_polyline": "ci...
2    {"id": "a15197835902", "summary_polyline": "s|...
3    {"id": "a15184146738", "summary_polyline": "s~...
4    {"id": "a15163453200", "summary_polyline": "wv...
..                                                 ...
608  {"id": "a2596458576", "summary_polyline": "{gp...
609  {"id": "a2589310060", "summary_polyline": "mho...
610  {"id": "a2583989594", "summary_polyline": "{mg...
611  {"id": "a2332672484", "summary_polyline": "sip...
612  {"id": "a2268997997", "summary_polyline": "sgp...

[613 rows x 1 columns]>

In [ ]:
# add decoded summary polylines
activities_df["polyline"] = activities_df["map"].apply(lambda m: polyline.decode(m["summary_polyline"]) if m and "summary_polyline" in m else [])


In [ ]:
activities_df.columns

Index(['resource_state', 'athlete', 'name', 'distance', 'moving_time',
       'elapsed_time', 'total_elevation_gain', 'type', 'sport_type',
       'workout_type', 'id', 'start_date', 'start_date_local', 'timezone',
       'utc_offset', 'location_city', 'location_state', 'location_country',
       'achievement_count', 'kudos_count', 'comment_count', 'athlete_count',
       'photo_count', 'map', 'trainer', 'commute', 'manual', 'private',
       'visibility', 'flagged', 'gear_id', 'start_latlng', 'end_latlng',
       'average_speed', 'max_speed', 'average_temp', 'average_watts',
       'device_watts', 'kilojoules', 'has_heartrate', 'average_heartrate',
       'max_heartrate', 'heartrate_opt_out', 'display_hide_heartrate_option',
       'elev_high', 'elev_low', 'upload_id', 'upload_id_str', 'external_id',
       'from_accepted_tag', 'pr_count', 'total_photo_count', 'has_kudoed',
       'average_cadence'],
      dtype='object')

In [ ]:
activities_df[['sport_type','average_speed', 'max_speed', 'average_temp']]

,sport_type,average_speed,max_speed,average_temp
0,Ride,6.543,11.229,24.0
1,Run,3.466,6.820,25.0
2,Run,3.370,7.160,23.0
3,TrailRun,3.104,7.000,26.0
4,Ride,7.591,14.520,27.0
...,...,...,...,...
607,Run,3.287,5.200,NaN
608,Run,2.915,3.900,NaN
609,Run,3.174,4.400,NaN
610,Run,2.992,4.800,NaN


In [ ]:
activities_df['id'].nunique()

612

In [ ]:
# Sélection des cinq premières lignes du DataFrame
first_five_activities = activities_df.head(5)

# Transposition du DataFrame pour afficher les colonnes comme des lignes
transposed_activities = first_five_activities.transpose()

# Affichage du DataFrame transposé
transposed_activities

## Data Cleaning
1. Supprimer colonnes non pertinentes
2. Renommer les colonnes pertinentes
3. Pour chaque colonne, adapter le contenu (temps, catégoriser avec les bons noms)
  A. Date en datetime
  B. temps en Km/h ET min/km
4. Ajouter une colonne qui combine Trail & CAP

In [ ]:
# Liste des colonnes à supprimer
columns_to_drop = [
    'resource_state', 'athlete', 'type', 'workout_type', 'utc_offset',
    'location_city', 'location_state', 'location_country', 'comment_count',
    'athlete_count', 'photo_count', 'trainer', 'commute', 'manual',
    'private', 'visibility', 'flagged', 'device_watts', 'heartrate_opt_out',
    'display_hide_heartrate_option', 'upload_id', 'upload_id_str', 'external_id',
    'from_accepted_tag', 'total_photo_count'
]

# Suppression des colonnes du DataFrame
activities_df_cleaned = activities_df.drop(columns=columns_to_drop)

# Affichage des premières lignes du DataFrame nettoyé
activities_df_cleaned.head()

,name,distance,moving_time,elapsed_time,total_elevation_gain,sport_type,id,start_date,start_date_local,timezone,...,average_watts,kilojoules,has_heartrate,average_heartrate,max_heartrate,elev_high,elev_low,pr_count,has_kudoed,average_cadence
0,Big moustache ?,37258.4,5694,6052,283.0,Ride,15210086165,2025-07-23T11:54:22Z,2025-07-23T13:54:22Z,(GMT+01:00) Europe/Brussels,...,118.3,673.5,True,109.1,137.0,141.0,57.4,3,False,NaN
1,Course à Dwight dans le Schrute,10017.0,2890,2893,75.0,Run,15197835902,2025-07-22T12:25:45Z,2025-07-22T14:25:45Z,(GMT+01:00) Europe/Brussels,...,NaN,NaN,True,140.1,160.0,90.4,56.2,0,False,73.2
2,Course à Kendrick le Lamar,14237.6,4225,4257,95.0,Run,15184146738,2025-07-21T07:18:50Z,2025-07-21T09:18:50Z,(GMT+01:00) Europe/Brussels,...,NaN,NaN,True,143.3,168.0,86.8,46.4,4,False,72.8
3,J’ai croisé un mec avec une enooooooorme mite,21892.7,7053,7117,573.0,TrailRun,15163453200,2025-07-19T08:01:22Z,2025-07-19T10:01:22Z,(GMT+01:00) Europe/Brussels,...,NaN,NaN,True,158.9,181.0,245.2,101.6,0,False,72.7
4,Pogacar dopé Gianetti pharmacien Mafia UCI,39428.6,5194,5194,345.0,Ride,15156172778,2025-07-18T14:14:46Z,2025-07-18T16:14:46Z,(GMT+01:00) Europe/Brussels,...,157.6,818.6,True,139.9,174.0,122.0,23.4,21,False,NaN


In [ ]:
activities_df_cleaned.columns

Index(['name', 'distance', 'moving_time', 'elapsed_time',
       'total_elevation_gain', 'sport_type', 'id', 'start_date',
       'start_date_local', 'timezone', 'achievement_count', 'kudos_count',
       'gear_id', 'start_latlng', 'end_latlng', 'average_speed', 'max_speed',
       'average_temp', 'average_watts', 'kilojoules', 'has_heartrate',
       'average_heartrate', 'max_heartrate', 'elev_high', 'elev_low',
       'pr_count', 'has_kudoed', 'average_cadence'],
      dtype='object')

In [ ]:
# Conversion de la colonne 'distance' de mètres en kilomètres
activities_df_cleaned['distance'] = activities_df_cleaned['distance'] / 1000

In [ ]:
# Définir la colonne 'id' comme index
activities_df_cleaned = activities_df_cleaned.set_index('id')


In [ ]:
activities_df_cleaned.info()

In [ ]:
# Conversion des colonnes 'moving_time' et 'elapsed_time' de secondes en minutes
activities_df_cleaned['moving_time'] = activities_df_cleaned['moving_time'] / 60
activities_df_cleaned['elapsed_time'] = activities_df_cleaned['elapsed_time'] / 60

In [ ]:
# Conversion de la colonne 'average_speed' de mètres par seconde en kilomètres par heure
activities_df_cleaned['average_speed'] = activities_df_cleaned['average_speed'] * 3.6
activities_df_cleaned['max_speed'] = activities_df_cleaned['max_speed'] * 3.6

In [ ]:
# Ajouter une nouvelle colonne 'minutes_per_km' qui convertit 'average_speed' en minutes par kilomètre
activities_df_cleaned['speed_minutes_per_km'] = 60 / activities_df_cleaned['average_speed']

In [ ]:
def convert_hours_to_hms(hours):
    # Convertir les heures en secondes
    total_seconds = int(hours * 3600)

    # Calculer les heures, minutes et secondes
    h = total_seconds // 3600
    remainder = total_seconds % 3600
    m = remainder // 60
    s = remainder % 60

    # Retourner le format HH:MM:SS
    return f"{h:02}:{m:02}:{s:02}"

# Ajouter une nouvelle colonne avec le format HH:MM:SS pour 'moving_time'
activities_df_cleaned['moving_time_hms'] = activities_df_cleaned['moving_time'].apply(convert_hours_to_hms)

# Ajouter une nouvelle colonne avec le format HH:MM:SS pour 'elapsed_time'
activities_df_cleaned['elapsed_time_hms'] = activities_df_cleaned['elapsed_time'].apply(convert_hours_to_hms)

In [ ]:
total_hours = activities_df['moving_time'].sum()
total_hours
convert_hours_to_hms(total_hours)

## Test PostgreSQL

In [ ]:
import psycopg2
from psycopg2 import sql
import pandas as pd

In [ ]:
df = pd.read_csv("/Users/arthurdercq/code/Data Science/Garmin_Dashboard/data/strava_activities_raw_2025-07-23.csv")

In [ ]:
import psycopg2

# Remplace ces informations par les tiennes
host = "localhost"
port = "5432"
dbname = "postgres"
user = "arthurdercq"
password = "qsdfgqsdfg"

try:
    conn = psycopg2.connect(
        host=host,
        port=port,
        dbname=dbname,
        user=user,
        password=password
    )
    print("Connexion réussie !")

    # Créer un curseur pour exécuter des requêtes
    cur = conn.cursor()
    cur.execute("SELECT version();")
    print("Version de PostgreSQL :", cur.fetchone())

    # N'oublie pas de fermer !
    cur.close()
    conn.close()

except Exception as e:
    print("Erreur de connexion :", e)


Connexion réussie !
Version de PostgreSQL : ('PostgreSQL 15.4 (Homebrew) on aarch64-apple-darwin22.4.0, compiled by Apple clang version 14.0.3 (clang-1403.0.22.14.1), 64-bit',)


In [ ]:
def store_df_in_postgresql(db_path):

    # Lire le fichier CSV dans un DataFrame pandas
    df = pd.read_csv(db_path)

    # Connexion à la base de données PostgreSQL
    # Remplacez les valeurs par vos informations de connexion PostgreSQL
    conn = psycopg2.connect(
        host = "localhost",
        database = "postgres",
        user = "arthurdercq",
        password= "qsdfgqsdfg",
        port = "5432"
    )


    # Créer un curseur pour exécuter des commandes SQL
    cur = conn.cursor()

    # Nom de la table dans laquelle vous allez importer les données
    table_name = "activities"

    # Créer la table dans PostgreSQL
    # Assurez-vous que les types de données correspondent à ceux de votre CSV
    create_table_query = sql.SQL("""
    CREATE TABLE IF NOT EXISTS {} (
    id SERIAL PRIMARY KEY,
    name VARCHAR(255),
    distance FLOAT,
    moving_time INTEGER,
    elapsed_time INTEGER,
    total_elevation_gain FLOAT,
    sport_type VARCHAR(255),
    start_date TIMESTAMP,
    start_date_local TIMESTAMP,
    timezone VARCHAR(50),
    achievement_count INTEGER,
    kudos_count INTEGER,
    gear_id VARCHAR(255),
    start_latlng VARCHAR(50),
    end_latlng VARCHAR(50),
    average_speed FLOAT,
    max_speed FLOAT,
    average_cadence FLOAT,
    average_temp FLOAT,
    has_heartrate BOOLEAN,
    average_heartrate FLOAT,
    max_heartrate FLOAT,
    elev_high FLOAT,
    elev_low FLOAT,
    pr_count INTEGER,
    has_kudoed BOOLEAN,
    average_watts FLOAT,
    kilojoules FLOAT
);
""").format(sql.Identifier(table_name))

    cur.execute(create_table_query)

    # Insérer les données dans la table
    for index, row in df.iterrows():
        insert_query = sql.SQL("""
        INSERT INTO {} (
            name, distance, moving_time, elapsed_time, total_elevation_gain,
            sport_type, start_date, start_date_local, timezone, achievement_count,
            kudos_count, gear_id, start_latlng, end_latlng, average_speed,
            max_speed, average_cadence, average_temp, has_heartrate,
            average_heartrate, max_heartrate, elev_high, elev_low, pr_count,
            has_kudoed, average_watts, kilojoules
        ) VALUES (
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
    )
        """).format(sql.Identifier(table_name))

        cur.execute(insert_query, (

        row['name'], row['distance'], row['moving_time'], row['elapsed_time'],
        row['total_elevation_gain'], row['sport_type'], row['start_date'],
        row['start_date_local'], row['timezone'], row['achievement_count'],
        row['kudos_count'], row['gear_id'], str(row['start_latlng']),
        str(row['end_latlng']), row['average_speed'], row['max_speed'],
        row['average_cadence'], row['average_temp'], row['has_heartrate'],
        row['average_heartrate'], row['max_heartrate'], row['elev_high'],
        row['elev_low'], row['pr_count'], row['has_kudoed'],
        row['average_watts'], row['kilojoules']
    ))

    # Valider les changements
    conn.commit()

    # Fermer le curseur et la connexion
    cur.close()
    conn.close()

    print("Données importées dans PostgreSQL ✅")

In [ ]:
store_df_in_postgresql(db_path="/Users/arthurdercq/code/Data Science/Garmin_Dashboard/data/strava_activities_raw_2025-07-23.csv")

Données importées dans PostgreSQL ✅


## Graphs

Qu'est ce que je veux montrer ? (Proposer une vision hebdo, mensuelle et annuelle)

- Volume hebomadaire
- D+ hebdo
- Répartition trail / CAP, vélo
- 


In [ ]:
def filtrer_trail_runs(df):
    # Colonnes à garder
    colonnes = ["name", "moving_time", "distance", "total_elevation_gain", "sport_type"]

    # Filtrer les colonnes si elles existent dans le DataFrame
    df = df[[col for col in colonnes if col in df.columns]]

    # Garder uniquement les lignes où sport_type est "TrailRun"
    df_filtré = df[df["sport_type"] == "TrailRun"]

    # Supprimer la colonne "sport_type" si tu ne veux plus l’avoir ensuite
    df_filtré = df_filtré.drop(columns="sport_type")

    return df_filtré


In [ ]:
def plot_elevations(df, output_file="../plots/elevations.png"):
    # Create a new figure
    plt.figure()

    # Compute activity start times (for facet ordering)
    start_times = (
        df.groupby("name").agg({"moving_time": "min"}).reset_index().sort_values("moving_time")
    )
    ncol = math.ceil(math.sqrt(len(start_times)))

    # Create facets
    p = sns.FacetGrid(
        data=df,
        col="name",
        col_wrap=ncol,
        col_order=start_times["name"],
        sharex=False,
        sharey=True,
    )

    # Add activities
    p = p.map(plt.plot, "distance", "total_elevation_gain", color="black", linewidth=4)

    # Update plot aesthetics
    p.set(xlabel=None)
    p.set(ylabel=None)
    p.set(xticks=[])
    p.set(yticks=[])
    p.set(xticklabels=[])
    p.set(yticklabels=[])
    p.set_titles(col_template="", row_template="")
    sns.despine(left=True, bottom=True)
    plt.subplots_adjust(left=0.05, bottom=0.05, right=0.95, top=0.95)
    plt.savefig(output_file)
    plt.close()

In [ ]:
de_plot_elev = filtrer_trail_runs(activities_df_cleaned)
plot_elevations(activities_df_cleaned)

In [ ]:
# select one activity
my_ride = activities_df_cleaned.iloc[0, :] # first activity (most recent)
# plot ride on map
centroid = [
    np.mean([coord[0] for coord in my_ride['map.polyline'][0]]),
    np.mean([coord[1] for coord in my_ride['map.polyline'][0]])
]
m = folium.Map(location=centroid, zoom_start=10)
folium.PolyLine(my_ride['map.polyline'], color='red').add_to(m)
display(m)

KeyError: 'map.polyline'